In [1]:
from confluent_kafka import Consumer, KafkaError
from fastavro import reader
import io
import json
import random
import settings

In [2]:
def make_stamp_annotation(record):
    r = {}
    classdict = {}
    maxprob = 0
    for k,v in record['probabilities'].items():
        classdict[k] = float('%.3f'%v)
        if v > maxprob:
            r['classification'] = k
            maxprob = v
    r['objectId']       = record['objectId']
    r['classdict']      = classdict
    if r['classification'] in ['VS', 'AGN', 'asteroid', 'bogus']:
        return None
    else:
        return r

In [3]:
def make_lc_annotation(record):
    r = {}
    r['objectId'] = record['oid']
    lcc = record['lc_classification']
    r['classification'] = lcc['class']
    classdict = {}
    for k,v in lcc['probabilities'].items():
        if v > 0.02:
            classdict[k] = float('%.3f'%v)
    r['classdict'] = classdict
    if r['classification'] in ['E', 'Periodic-Other']:
        return None
    else:
        return r

In [4]:
print(settings.ALERCE_KAFKA)
print(settings.ALERCE_GROUP_ID)
print(settings.ALERCE_PASSWORD)

b-2-public.publicproduction.o8ncxm.c18.kafka.us-east-1.amazonaws.com:9196,b-1-public.publicproduction.o8ncxm.c18.kafka.us-east-1.amazonaws.com:9196,b-3-public.publicproduction.o8ncxm.c18.kafka.us-east-1.amazonaws.com:9196
lasair003
ZTAyZjdlNjAwNmU4


In [ ]:
conf = {
    'bootstrap.servers': settings.ALERCE_KAFKA,
    'group.id'         : settings.ALERCE_GROUP_ID,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanism'   : 'SCRAM-SHA-512',
    'sasl.username'    : 'lasair',
    'sasl.password'    : settings.ALERCE_PASSWORD,
    'auto.offset.reset': 'earliest',
}
streamReader = Consumer(conf)
t = list(streamReader.list_topics().topics.keys())
print('Topics are ', t)

%4|1725873630.040|FAIL|rdkafka#consumer-1| [thrd:sasl_ssl://b-2-public.publicproduction.o8ncxm.c18.kafka.us-east]: sasl_ssl://b-2-public.publicproduction.o8ncxm.c18.kafka.us-east-1.amazonaws.com:9196/bootstrap: Connection setup timed out in state CONNECT (after 30129ms in state CONNECT)
%4|1725873630.970|FAIL|rdkafka#consumer-1| [thrd:sasl_ssl://b-1-public.publicproduction.o8ncxm.c18.kafka.us-east]: sasl_ssl://b-1-public.publicproduction.o8ncxm.c18.kafka.us-east-1.amazonaws.com:9196/bootstrap: Connection setup timed out in state CONNECT (after 30055ms in state CONNECT)
%4|1725873631.972|FAIL|rdkafka#consumer-1| [thrd:sasl_ssl://b-3-public.publicproduction.o8ncxm.c18.kafka.us-east]: sasl_ssl://b-3-public.publicproduction.o8ncxm.c18.kafka.us-east-1.amazonaws.com:9196/bootstrap: Connection setup timed out in state CONNECT (after 30051ms in state CONNECT)
%4|1725873660.079|FAIL|rdkafka#consumer-1| [thrd:sasl_ssl://b-2-public.publicproduction.o8ncxm.c18.kafka.us-east]: sasl_ssl://b-2-public